In [113]:

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_4144cbc03d8e478aa7890f51134abd45(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', 'c8c3a94f3b384de3bfb123dca37f5455')
    hconf.set(prefix + '.username', '07ac97106bd9416e93058c183c4e40db')
    hconf.set(prefix + '.password', 'ir(30c.hp}QdpYdu')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_4144cbc03d8e478aa7890f51134abd45(name)

In [114]:
scores = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://CS340." + name + "/genome-scores.csv")

In [115]:
tag_names = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://CS340." + name + "/genome-tags.csv")

In [116]:

movies = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://CS340." + name + "/movies.csv")

In [117]:

ratings = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://CS340." + name + "/ratings.csv")

In [118]:

tags = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://CS340." + name + "/tags.csv")

In [119]:
"""PART I - THE MOST RELIABLE USERS"""

'PART I - THE MOST RELIABLE USERS'

In [123]:
from pyspark.sql.functions import tanh
temp_p1_1 = scores.join(tags.join(tag_names, "tag").select("userId", "movieId", "tagId"), ["tagId", "movieId"])
temp_p1_1.cache()

DataFrame[tagId: int, movieId: int, relevance: double, userId: int]

In [125]:
temp_p1_2 = temp_p1_1.select("userId", tanh(temp_p1_1.relevance*20-12))
temp_p1_2 = temp_p1_2.groupBy("userId").sum("TANH(((relevance * 20) - 12))").withColumnRenamed("sum(TANH(((relevance * 20) - 12)))", "reliability_score")
temp_p1_2.cache()

DataFrame[userId: int, reliability_score: double]

In [134]:
mostReliableUsers = temp_p1_2.sort("reliability_score", ascending=False).limit(1000)
mostReliableUsers.cache()
mostReliableUsers.show(1000)

+------+------------------+
|userId| reliability_score|
+------+------------------+
| 88738|5886.2286052266945|
| 58612| 1700.314210560726|
| 52814|1648.8382288692462|
| 11081|1582.5985838545298|
| 10616|1447.9943988488765|
|  9815|1413.2447981826153|
| 68558| 1261.188971907667|
|  1741| 1119.254178487934|
| 25737|1068.4360438182057|
|  4450|1000.3835964912715|
| 27898| 969.4690500707221|
|122523| 961.2716671956215|
|119367| 959.6825546999671|
| 77463| 927.4315434523479|
|124998| 854.3481069572274|
| 77297| 845.1531603150677|
| 28906| 840.9469449647917|
|120937|  797.676160294742|
|  6431|  665.882176508323|
|111982|    662.9396577253|
|132195| 662.4140364061682|
| 10514| 639.5480101354966|
|107711| 607.9753558248359|
|131900| 591.0842834020634|
| 20845| 577.0503608958549|
| 28780| 575.6579425139817|
| 57434|  566.768620753232|
|130446| 553.6307449052152|
| 72766| 544.8235867476017|
| 49129| 544.6906064447297|
| 68625| 535.7112386908603|
| 42245| 531.4504395240612|
| 96370|  528.479420

In [127]:
"""PART 2 - BEST MOVIES"""

'PART 2 - BEST MOVIES'

In [128]:
temp_p2_1 = ratings.join(mostReliableUsers, "userId")
temp_p2_1 = temp_p2_1.select("userId", "movieId", temp_p2_1.rating*temp_p2_1.reliability_score)
temp_p2_1 = temp_p2_1.withColumnRenamed("(rating * reliability_score)" ,"reliable_score")
temp_p2_1.cache()

DataFrame[userId: int, movieId: int, reliable_score: double]

In [133]:
temp_p2_2 = temp_p2_1.groupBy("movieId").sum("reliable_score")
temp_p2_2 = temp_p2_2.withColumnRenamed("sum(reliable_score)", "score").sort("score", ascending=False).limit(20)
temp_p2_2.cache()

DataFrame[movieId: int, score: double]

In [130]:
bestMovies = temp_p2_2.join(movies, "movieId").select("movieId", "title", "score").sort("score", ascending=False)
bestMovies.cache()

DataFrame[movieId: int, title: string, score: double]

In [131]:
bestMovies.show()

+-------+--------------------+------------------+
|movieId|               title|             score|
+-------+--------------------+------------------+
|   2571|  Matrix, The (1999)| 471354.5594368033|
|   2959|   Fight Club (1999)| 426152.0577531982|
|    296| Pulp Fiction (1994)|415367.08530751517|
|    318|Shawshank Redempt...|409525.17213523336|
|    356| Forrest Gump (1994)| 395281.5232354894|
|   4993|Lord of the Rings...| 395052.4140476214|
|   4226|      Memento (2000)|378664.83073142625|
|  79132|    Inception (2010)|375745.96609234525|
|   5952|Lord of the Rings...| 372978.4285739493|
|   7153|Lord of the Rings...|364419.68578229164|
|  58559|Dark Knight, The ...| 361753.2328497147|
|    593|Silence of the La...|360313.65050738544|
|    260|Star Wars: Episod...|359782.00395323965|
|   2858|American Beauty (...| 358955.4184849296|
|   1270|Back to the Futur...| 352814.6100077705|
|   7361|Eternal Sunshine ...| 351084.1205756393|
|   1196|Star Wars: Episod...|349536.59739373694|


In [132]:
#FATIH CAGATAY GULMEZ 
#213962062